In [13]:
from pypdf import PdfReader
import pandas as pd

In [3]:
# read the file you're working with
reader = PdfReader('Képviselők_20250228.pdf')

In [4]:
print(len(reader.pages))

2258


In [157]:
# to inspect one page's text

page = reader.pages[1]
text = page.extract_text()
print(text)

A) rész
VAGYONI NYILATKOZAT
I. Ingatlanok (kivéve a nyilatkozatot adó, valamint a vele közös háztartásban élő
házastársa vagy élettársa és gyermeke(i) kizárólagos használatára fenntartott ingatlant)2
1. Ingatlan:
a) A település neve, ahol az ingatlan fekszik (Budapesten kerület is):
Veszprém
b) Az ingatlan területnagysága:
72 m2
c) Művelési ága (vagy a művelés alól kivett terület elnevezése):
-----
d) Az épület fő rendeltetés szerinti jellege (lakóház, üdülő, gazdasági épület stb.), az épület
alapterülete:
lakóház  77 m2
e) Az ingatlan jogi jellege (családi ház, társasház, szövetkezeti ház, műemlék, műhely, üzlet,
műterem, rendelő, garázs, bányatelek stb.):
társasház
f) A nyilatkozó jogállása (tulajdonos, állandó, illetve tartós használó, haszonélvező, bérlő stb.):
tulajdonos
g) Közös tulajdon esetén a tulajdoni hányad mértéke: 1/2
h) A szerzés jogcíme, ideje (a jogviszony kezdete):   2019. november 7.
2. Ingatlan:
a) A település neve, ahol az ingatlan fekszik (Budapesten kerület is):


In [150]:
# Define a dataframe to store your structured data

vny_2025 = pd.DataFrame(columns = ['oldal', 'sor', 'PID', 'name', 'ing_sz', 'ingatlan', 'terület', 'művelési ág', 'rendeltetés', 'alapterület', 'jogi jelleg', 
                                  'jogállás', 'tulajdoni hányad', 'jogviszony'])


In [154]:
# to track which lines to write in
counter = 0

# Person ID - to track individuals
pid = int(0)


for i in range(len(reader.pages)):

    page = reader.pages[i]
    text = page.extract_text()
    t_lines = list(text.split('\n'))
    
    # to reach back and forward on the page if needed
    linecount = 0
    
    for line in t_lines:
        linecount += 1
        if '2. A nyilatkozatot adó neve:' in line:
            counter += 1
            pid += 1
            ing_sz = 0
            name = str(line.removeprefix('2. A nyilatkozatot adó neve: ').strip())
            vny_2025.loc[counter, 'name'] = name
            vny_2025.loc[counter, 'PID'] = int(pid)
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)
            
        if 'a) A település neve, ahol az ingatlan fekszik' in line:
            ing_sz += 1
            ing = t_lines[linecount]
            if ing_sz == 1:
                vny_2025.loc[counter, 'ingatlan'] = ing
                vny_2025.loc[counter, 'ing_sz'] = ing_sz
                vny_2025.loc[counter, 'oldal'] = int(i+1)
                vny_2025.loc[counter, 'sor'] = int(linecount)
                
            elif ing_sz > 1:
                counter += 1
                vny_2025.loc[counter, 'name'] = name
                vny_2025.loc[counter, 'PID'] = int(pid)
                vny_2025.loc[counter, 'ingatlan'] = ing
                vny_2025.loc[counter, 'ing_sz'] = ing_sz
                vny_2025.loc[counter, 'oldal'] = int(i+1)
                vny_2025.loc[counter, 'sor'] = int(linecount)
                
                
        if 'b) Az ingatlan területnagysága:' in line:
            nm = t_lines[linecount]
            vny_2025.loc[counter, 'terület'] = nm
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

        if 'c) Művelési ága (vagy a művelés alól' in line:
            mua = t_lines[linecount]
            vny_2025.loc[counter, 'művelési ág'] = mua
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)
            
        if 'd) Az épület fő rendeltetés szerinti jellege' in line:
            rend = t_lines[linecount+1]
            if '-' in rend:
                continue
            else:
                rend = rend.split()
            vny_2025.loc[counter, 'rendeltetés'] = rend[0]
            try:
                vny_2025.loc[counter, 'alapterület'] = rend[1]
            except:
                vny_2025.loc[counter, 'rendeltetés'] = ''
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

        if 'e) Az ingatlan jogi jellege' in line:
            jj = t_lines[linecount+1]
            vny_2025.loc[counter, 'jogi jelleg'] = jj
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

        if 'f) A nyilatkozó jogállása (tulajdonos' in line:
            jog = t_lines[linecount]
            vny_2025.loc[counter, 'jogállás'] = jog
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

        if 'g) Közös tulajdon esetén a tulajdoni hányad' in line:
            tulh = str(line.removeprefix('g) Közös tulajdon esetén a tulajdoni hányad mértéke: '))
            vny_2025.loc[counter, 'tulajdoni hányad'] = tulh
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

        if 'h) A szerzés jogcíme, ideje (a jogviszony' in line:
            jogvisz = str(line.removeprefix('h) A szerzés jogcíme, ideje (a jogviszony kezdete): '))
            vny_2025.loc[counter, 'jogviszony'] = jogvisz
            vny_2025.loc[counter, 'oldal'] = int(i+1)
            vny_2025.loc[counter, 'sor'] = int(linecount)

In [156]:
# save the file
vny_2025.to_csv('2025_sorted.csv', index = False)